# Q-러닝(Q-Learning)으로 FrozenLake 문제 풀기

**학습 목표:**
- 에이전트가 환경과 상호작용하며 보상을 통해 학습하는 **강화학습**의 기본 개념을 이해합니다.
- 대표적인 가치 기반(Value-based) 알고리즘인 **Q-러닝**의 동작 원리를 학습합니다.
- 상태(State)와 행동(Action)의 가치를 저장하는 **Q-테이블**을 만들고, **벨만 방정식**을 통해 이를 업데이트하는 과정을 구현합니다.
- 탐험(Exploration)과 활용(Exploitation)의 균형을 맞추는 **Epsilon-Greedy** 전략을 적용합니다.
- **OpenAI Gym**의 **FrozenLake** 환경에서 미끄러운 얼음판을 건너는 에이전트를 성공적으로 훈련시킵니다.

In [ ]:
# gym 라이브러리가 없다면 설치합니다.
!pip install gym

In [ ]:
import gym
import numpy as np
import random
import time
from IPython.display import clear_output

### (1) 환경(Environment) 생성
FrozenLake-v1: 4x4 크기의 그리드 월드 환경.
- `S`: 시작 지점
- `F`: 얼어있는 길 (안전)
- `H`: 구멍 (빠지면 -1 보상, 에피소드 종료)
- `G`: 목표 지점 (도착하면 +1 보상, 에피소드 종료)

In [ ]:
env = gym.make('FrozenLake-v1', is_slippery=True) # is_slippery=True: 미끄러운 환경
state_space_size = env.observation_space.n
action_space_size = env.action_space.n

print(f"State(상태) 공간 크기: {state_space_size}")
print(f"Action(행동) 공간 크기: {action_space_size}")

### (2) Q-테이블 초기화 및 하이퍼파라미터 설정
Q-테이블은 모든 상태-행동 쌍에 대한 가치(Q-value)를 저장하는 행렬입니다. 처음에는 모든 값을 0으로 초기화하고, 학습을 통해 점차 최적의 값으로 업데이트해 나갑니다.

In [ ]:
q_table = np.zeros((state_space_size, action_space_size))

# 하이퍼파라미터
total_episodes = 20000        # 총 학습 에피소드 수
learning_rate = 0.1           # 학습률 (alpha)
gamma = 0.99                  # 할인율 (미래 보상의 가치를 현재 가치로 환산하는 비율)

epsilon = 1.0                 # 탐험(Exploration) 확률
max_epsilon = 1.0             
min_epsilon = 0.01            
epsilon_decay_rate = 0.0001   # Epsilon 감소율 (더 많은 탐험을 위해 천천히 감소)

### (3) Q-러닝 알고리즘 학습
에이전트는 각 타임스텝마다 Epsilon-Greedy 전략에 따라 행동을 선택하고, 환경으로부터 다음 상태(new_state)와 보상(reward)을 받습니다. 이 정보를 바탕으로 아래의 벨만 방정식을 사용하여 Q-테이블을 업데이트합니다.
$$ Q(s, a) \leftarrow Q(s, a) + \alpha [R(s, a) + \gamma \max_{a'} Q(s', a') - Q(s, a)] $$

In [ ]:
rewards = []

for episode in range(total_episodes):
    state = env.reset()
    done = False
    episode_rewards = 0
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # 탐험: 무작위 행동 선택
        else:
            action = np.argmax(q_table[state, :]) # 활용: Q-값이 가장 높은 행동 선택
            
        new_state, reward, done, info = env.step(action)
        
        # Q-테이블 업데이트
        old_value = q_table[state, action]
        next_max = np.max(q_table[new_state, :])
        new_value = old_value + learning_rate * (reward + gamma * next_max - old_value)
        q_table[state, action] = new_value
        
        state = new_state
        episode_rewards += reward
        
    # Epsilon 감소 (탐험 확률을 점차 줄여나감)
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-epsilon_decay_rate * episode)
    rewards.append(episode_rewards)

print("Training finished.\n")
print("--- Learned Q-Table ---")
print(q_table)

### (4) 학습 결과 분석 및 시각화
학습이 진행됨에 따라 에피소드당 평균 보상이 증가하는지 확인하여 학습이 잘 이루어졌는지 평가합니다.

In [ ]:
# 보상 그래프 시각화
rewards_per_thousand_episodes = np.split(np.array(rewards), total_episodes / 1000)
count = 1000
print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(f"{count}: {sum(r/1000)}")
    count += 1000

# 학습된 정책으로 에이전트 실행
for episode in range(3):
    state = env.reset()
    done = False
    print(f"\n--- EPISODE {episode+1} ---")
    time.sleep(1)
    
    for step in range(100):
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        
        action = np.argmax(q_table[state, :])
        new_state, reward, done, info = env.step(action)
        
        if done:
            env.render()
            if reward == 1:
                print("****GOAL!****")
                time.sleep(2)
            break
        state = new_state
        
env.close()